<a href="https://colab.research.google.com/github/shadowluigi27/Projects-in-ML-F22/blob/main/Projects_in_ML_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 1
Some websites I used to help with getting my NN working

https://www.tensorflow.org/guide/autodiff
(provided)

https://www.tensorflow.org/api_docs/python/tf/GradientTape
(provided)

https://scikit-learn.org/stable/modules/neural_networks_supervised.html

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html 

I wanted to use Skitlearn since I used it in my last HW, and learning the different types of models in one library list would allow me to more effectivly use it

#Task 2

In [ ]:
# import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import tensorflow as tf


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense



Dataset link: https://archive.ics.uci.edu/ml/datasets/census+income 

EDA of dataset below


In [ ]:
df = pd.read_csv('adult.csv')
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [ ]:
df.shape

(32561, 15)

In [ ]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education.num      int64
marital.status    object
occupation        object
relationship      object
race              object
sex               object
capital.gain       int64
capital.loss       int64
hours.per.week     int64
native.country    object
income            object
dtype: object

In [ ]:
df.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

no non null values, just data with ?, so we will remove rows with those values

From research: https://www.kaggle.com/datasets/uciml/adult-census-income/discussion/32698

Fnlweight appears to be different for each state, and just represents # of people who share similar traits.

In [ ]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

In [ ]:
#rename columns so we can look at values better
df = df.rename(columns={"education.num": "educationNum", 
                   "marital.status": "maritalStatus",
                   "capital.gain" : "capitalGain",
                   "capital.loss" : "capitalLoss",
                   "hours.per.week" : "capitalPerWeek",
                   "native.country" : "nativeCountry"})

In [ ]:
labels = ["workclass","maritalStatus","occupation","relationship","race","sex","nativeCountry"]

for label in labels:
  df = df.loc[df[label] != "?"]

df.shape

(30162, 15)

In [ ]:
#df = df.drop('fnlwgt',axis=1)

In [ ]:
df.describe()

,age,fnlwgt,educationNum,capitalGain,capitalLoss,capitalPerWeek
count,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000
mean,38.437902,1.897938e+05,10.121312,1092.007858,88.372489,40.931238
std,13.134665,1.056530e+05,2.549995,7406.346497,404.298370,11.979984
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.176272e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.784250e+05,10.000000,0.000000,0.000000,40.000000
75%,47.000000,2.376285e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
df.describe(include=['O'])

,workclass,education,maritalStatus,occupation,relationship,race,sex,nativeCountry,income
count,30162,30162,30162,30162,30162,30162,30162,30162,30162
unique,7,16,7,14,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,22286,9840,14065,4038,12463,25933,20380,27504,22654


##Preprocessing Data

Upon research, fnlwgt is just how many people fall under those catagories, so we should be able to disregard that in our model, since other feaures determine the income, not how many people fall in this catagory

https://www.kaggle.com/datasets/uciml/adult-census-income/discussion/32698


In [ ]:
df = df.drop('fnlwgt', axis=1)

We can assign gender and income into a numerical value

In [ ]:
df.sex = df.sex.map({'Male': 0, 'Female': 1 })
df.income = df.income.map({'<=50K': 0, '>50K': 1 })

In [ ]:
df.income.value_counts(normalize=True)

0    0.751078
1    0.248922
Name: income, dtype: float64

For the rest of the values, we can use one hot encoding.

##workclass

In [ ]:
df.workclass.value_counts()

Private             22286
Self-emp-not-inc     2499
Local-gov            2067
State-gov            1279
Self-emp-inc         1074
Federal-gov           943
Without-pay            14
Name: workclass, dtype: int64

There seems to be a good distrubution of work classes

##education

In [ ]:
df.education.value_counts()

HS-grad         9840
Some-college    6678
Bachelors       5044
Masters         1627
Assoc-voc       1307
11th            1048
Assoc-acdm      1008
10th             820
7th-8th          557
Prof-school      542
9th              455
12th             377
Doctorate        375
5th-6th          288
1st-4th          151
Preschool         45
Name: education, dtype: int64

In [ ]:
#16 education values, let us keep the ones that most places use as filters
df.loc[~df.education.isin(['HS-grad', 'Bachelors', 'Masters', 'Prof-school','Doctorate']), 'education'] = "uneducated"

##maritial status

In [ ]:
df.maritalStatus.value_counts()

Married-civ-spouse       14065
Never-married             9726
Divorced                  4214
Separated                  939
Widowed                    827
Married-spouse-absent      370
Married-AF-spouse           21
Name: maritalStatus, dtype: int64

There seems to be a good distrubution of martial status

##occupation

In [ ]:
df.occupation.value_counts()

Prof-specialty       4038
Craft-repair         4030
Exec-managerial      3992
Adm-clerical         3721
Sales                3584
Other-service        3212
Machine-op-inspct    1966
Transport-moving     1572
Handlers-cleaners    1350
Farming-fishing       989
Tech-support          912
Protective-serv       644
Priv-house-serv       143
Armed-Forces            9
Name: occupation, dtype: int64

Although there are many values for occupation, this is important to determine if salary is high

Relationship,  race, sex have less than 7 values, so it is unesscary to join them more

##Native Country

In [ ]:
df.nativeCountry.value_counts()

United-States                 27504
Mexico                          610
Philippines                     188
Germany                         128
Puerto-Rico                     109
Canada                          107
El-Salvador                     100
India                           100
Cuba                             92
England                          86
Jamaica                          80
South                            71
Italy                            68
China                            68
Dominican-Republic               67
Vietnam                          64
Guatemala                        63
Japan                            59
Poland                           56
Columbia                         56
Iran                             42
Taiwan                           42
Haiti                            42
Portugal                         34
Nicaragua                        33
Peru                             30
Greece                           29
Ecuador                     

In [ ]:
#there is a majority of Americans in the dataset, so let us split this based on whether they are in or not in the US
df.loc[~df.nativeCountry.isin(['United-States']), 'nativeCountry'] = "Out-of-States"
df.head()


,age,workclass,education,educationNum,maritalStatus,occupation,relationship,race,sex,capitalGain,capitalLoss,capitalPerWeek,nativeCountry,income
1,82,Private,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,1,0,4356,18,United-States,0
3,54,Private,uneducated,4,Divorced,Machine-op-inspct,Unmarried,White,1,0,3900,40,United-States,0
4,41,Private,uneducated,10,Separated,Prof-specialty,Own-child,White,1,0,3900,40,United-States,0
5,34,Private,HS-grad,9,Divorced,Other-service,Unmarried,White,1,0,3770,45,United-States,0
6,38,Private,uneducated,6,Separated,Adm-clerical,Unmarried,White,0,0,3770,40,United-States,0


Performing 1 hot encoding

In [ ]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [ ]:
features_to_encode = ['workclass', 'education', 'maritalStatus','occupation','relationship','race','nativeCountry']

for feature in features_to_encode:
    df = encode_and_bind(df, feature)
df.head()

,age,educationNum,sex,capitalGain,capitalLoss,capitalPerWeek,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,nativeCountry_Out-of-States,nativeCountry_United-States
1,82,9,1,0,4356,18,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
3,54,4,1,0,3900,40,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1
4,41,10,1,0,3900,40,0,0,0,1,...,1,0,0,0,0,0,0,1,0,1
5,34,9,1,0,3770,45,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1
6,38,6,0,0,3770,40,0,0,0,1,...,0,1,0,0,0,0,0,1,0,1


In [ ]:
df.shape

(30162, 54)

##train test  and Nureal Network creation

In [ ]:
#train test split
input = df.drop(['income'], axis=1)
target = df.income

MMScaler = MinMaxScaler()
input = MMScaler.fit_transform(input)

x_train,x_test,y_train,y_test = train_test_split(input,target,test_size= .25,random_state = 1)

We will implement a Multi-layered-perceptrion (in our case, 2 layers only) with Sklearn, which uses backpropagation \

In [ ]:
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(hidden_layer_sizes = (50), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = True
    )
NN.fit(x_train,y_train)

Iteration 1, loss = 0.39744453
Iteration 2, loss = 0.36187519
Iteration 3, loss = 0.35553342
Iteration 4, loss = 0.35504572
Iteration 5, loss = 0.34802471
Iteration 6, loss = 0.34602822
Iteration 7, loss = 0.34518929
Iteration 8, loss = 0.34044204
Iteration 9, loss = 0.34046656
Iteration 10, loss = 0.33840083
Iteration 11, loss = 0.34059668
Iteration 12, loss = 0.34246049
Iteration 13, loss = 0.33456596
Iteration 14, loss = 0.33490715
Iteration 15, loss = 0.33171217
Iteration 16, loss = 0.32933607
Iteration 17, loss = 0.33369604
Iteration 18, loss = 0.32935630
Iteration 19, loss = 0.33034579
Iteration 20, loss = 0.32618158
Iteration 21, loss = 0.32761129
Iteration 22, loss = 0.32647170
Iteration 23, loss = 0.32464220
Iteration 24, loss = 0.32743506
Iteration 25, loss = 0.32497949
Iteration 26, loss = 0.32859802
Iteration 27, loss = 0.32132275
Iteration 28, loss = 0.32312169
Iteration 29, loss = 0.32230785
Iteration 30, loss = 0.32310254
Iteration 31, loss = 0.32126494
Iteration 32, los

MLPClassifier(alpha=0.001, hidden_layer_sizes=50, learning_rate_init=0.9,
              solver='sgd', verbose=True)

In [ ]:
y_predict = NN.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
accuracy

0.8386155682270255

#Task 3

My initial hyperparameters are above. I used 50 nodes on the first hidden layer to approximatly match the number of attributes, with the standard learning rate of .9 as a baseline

As you can see, the initial accuracy with mostly standard is a respectable 83% accuracy. 

Below are some variations I thought of either while creating the initial hyperparameters, or after finishing my first implementation of the Nueral Networks

(I have turned off the verbose feature as to prevent unesscary printing)

In [ ]:
#Variation 1) use Adam instead of SGD
NN1 = MLPClassifier(
    hidden_layer_sizes = (50), 
    activation = 'relu',
    solver = 'adam',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    )
NN1.fit(x_train,y_train)
y_predict = NN1.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.7479114175838748


In [54]:
#Variation 2) use a different activation function
NN2 = MLPClassifier(
    hidden_layer_sizes = (50), 
    activation = 'logistic',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    )
NN2.fit(x_train,y_train)
y_predict = NN2.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.8448481633735578


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [55]:
#Variation 3) Adam + different activation function
NN3 = MLPClassifier(
    hidden_layer_sizes = (50), 
    activation = 'logistic',
    solver = 'adam',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    )
NN3.fit(x_train,y_train)
y_predict = NN3.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.821111258453786


In [ ]:
#Variation 4) 2 layers, 50,25
NN4 = MLPClassifier(
    hidden_layer_sizes = (50,25), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    )
NN4.fit(x_train,y_train)
y_predict = NN4.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.7708526720594086


In [ ]:
#Variation 5) 2 layers, learning rate is lower
NN5 = MLPClassifier(
    hidden_layer_sizes = (50), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .6,
    max_iter = 200,
    verbose = False
    )
NN5.fit(x_train,y_train)
y_predict = NN5.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)


0.8371568757459223


In [ ]:
#Variation 6) 2 layers, 50-> 25 nodes
NN5 = MLPClassifier(
    hidden_layer_sizes = (50,25), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    ) 
NN5.fit(x_train,y_train)
y_predict = NN5.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.8364938337090572


In [ ]:
#Variation 7) 4 layers, each 12 nodes
NN7 = MLPClassifier(
    hidden_layer_sizes = (12,12,12,12), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    )
NN7.fit(x_train,y_train)
y_predict = NN7.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.7479114175838748


In [ ]:
#Variation 8) 4 layers, 12 nodes, alpha is lower (2 + 4)
NN8 = MLPClassifier(
    hidden_layer_sizes = (12,12,12,12), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .6,
    max_iter = 200,
    verbose = False
    )
NN8.fit(x_train,y_train)
y_predict = NN8.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.7479114175838748


In [ ]:
#Variation 9) 4 layers, 50 -> 25 -> 12 -> 6
NN9 = MLPClassifier(
    hidden_layer_sizes = (50,25,12,6), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    )
NN9.fit(x_train,y_train)
y_predict = NN9.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.7479114175838748


In [ ]:
#Variation 10) 2 layers, node count is 25
NN10 = MLPClassifier(
    hidden_layer_sizes = (25), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    )
NN10.fit(x_train,y_train)
y_predict = NN10.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.8417981700039783


In [ ]:
#Alteration 11) 2 layers, node count is 65
NN11 = MLPClassifier(
    hidden_layer_sizes = (65), 
    activation = 'relu',
    solver = 'sgd',
    alpha = .001,
    batch_size = 'auto',
    learning_rate_init = .9,
    max_iter = 200,
    verbose = False
    )
NN11.fit(x_train,y_train)
y_predict = NN11.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.8431242540777085


Here is a compliation of the accuracy score using Alternatives:

------------------------------------------
Orginal Accuracy:83.86%

-------------------------

1) Accuracy: 74.79%

2) Accuracy: 84.69%

3) Accuracy:76.84%

4) Accuracy:77.08%

5) Accuracy: 83.715

6) Accuracy:83.64%

7) Accuracy: 74.79%

8) Accuracy:74.79%

9) Accuracy: 74.79%

10) Accuracy: 84.17%

11) Accuracy:84.3%

The issue with the models with multiple layers, is that the data is mostly likely overfitting on the training data, and thus is performing worse on the test data.

The activation function being a sigmoid instead of relu didn't affect much, but the change from scholastic gradient descent to adam had a huge negative impact on the accuracy of the model



#Task 4 
###Using another model 

I will use linear regression model on the same problem and compare the accuracy on this dataset to the accuracy of our Neural Networks (and the variations  

In [59]:
Regress = LogisticRegression(max_iter = 200)

Regress.fit(x_train,y_train)

y_predict = Regress.predict(x_test)

accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.8460416390399151


As we can see, logistic regression has a very comparable accuracy level to the higher end of Neural networks. The only change made to the regression is that I increased the maximum Iteration to match those of the Neural Networks inital maximum epochs. Without it, the regression does not coverge by the time the 100th iteration is ran.

In this case, there isn't as many features that Logistic Regression cannot handle compared to Neural networks, but it does take more iterations to acheive the same level of accuracy, which is a big consideration in datasets with much more information.